In [1]:
!pip install langchain openai google-cloud-aiplatform
!pip install langchain
!pip install openai==0.28
from langchain.llms import openai
import random
!pip install langchain
import langchain
import os
from langchain.llms import OpenAI
!pip install local
!pip install gdown
import gdown
import pandas as pd

  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/e7/78/77d4d56bb29b9bbe9ec80410bb32c241fcd8daee4bb319ce8a925e48487b/google_cloud_aiplatform-1.48.0-py2.py3-none-any.whl.metadata
  Using cached google_cloud_aiplatform-1.48.0-py2.py3-none-any.whl.metadata (30 kB)
  Obtaining dependency information for google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<3.0.0dev,>=1.34.1 from https://files.pythonhosted.org/packages/86/75/59a3ad90d9b4ff5b3e0537611dbe885aeb96124521c9d35aa079f1e0f2c9/google_api_core-2.18.0-py3-none-any.whl.metadata
  Using cached google_api_core-2.18.0-py3-none-any.whl.metadata (2.7 kB)
  Obtaining dependency information for google-cloud-storage<3.0.0dev,>=1.32.0 from https://files.pythonhosted.org/packages/cb/e5/7d045d188f4ef85d94b9e3ae1bf876170c6b9f4c9a950124978efc36f680/google_cloud_storage-2.16.0-py2.py3-none-any.whl.metadata
  Using cached google_cloud_storage-2.16.0-py2.py3-none

In [46]:
file_path = "TEAM/movies_merged.csv"

movies_df = pd.read_csv(file_path)

print(movies_df.head())

       director_name  duration      actor_2_name  \
0      James Cameron     178.0  Joel David Moore   
1     Gore Verbinski     169.0     Orlando Bloom   
2         Sam Mendes     148.0      Rory Kinnear   
3  Christopher Nolan     164.0    Christian Bale   
4     Andrew Stanton     132.0   Samantha Morton   

                          genres_x     actor_1_name  \
0  Action|Adventure|Fantasy|Sci-Fi      CCH Pounder   
1         Action|Adventure|Fantasy      Johnny Depp   
2        Action|Adventure|Thriller  Christoph Waltz   
3                  Action|Thriller        Tom Hardy   
4          Action|Adventure|Sci-Fi     Daryl Sabara   

                              movie_title          actor_3_name  \
0                                  Avatar             Wes Studi   
1  Pirates of the Caribbean At Worlds End        Jack Davenport   
2                                 Spectre      Stephanie Sigman   
3                   The Dark Knight Rises  Joseph Gordon-Levitt   
4                    

In [50]:
import requests
import json

def extract_message_oai(response_data):
    # Check if the response_data contains the 'choices' key
    if 'choices' in response_data:
        # Extract the first choice
        first_choice = response_data['choices'][0]
        # Check if the choice contains a 'message' field
        if 'message' in first_choice:
            message = first_choice['message']
            # Check if the message contains the recommendation
            if 'content' in message:
                return message['content']
    return "No recommendation found"


def get_recommendation(genre, movies_df):
    url = "https://llm.api.ai8.io/query_llm"
    api_key = 'iBaXjSsgwr5umhLoAEa5BO1ZLhy62JIyMxH-x9ebG6w'
    model = "gpt-3.5-turbo-0125"

    filtered_movies = movies_df[movies_df['genres_x'].str.contains(genre.capitalize())]

    if len(filtered_movies) == 0:
        return "No movies found for the specified genre."
    else:
        random_movie = filtered_movies.sample(n=1)
        movie_to_recommend = random_movie['movie_title'].iloc[0]

        data = {
            "model": model,
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": f"If I like {genre} movies from the dataset, recommend me a movie."}
            ]
        }

        headers = {'Authorization': api_key}

        response = requests.post(url, json=data, headers=headers)
        print("Response data:", response.content)

        if response.status_code == 200:
            response_data = json.loads(response.content)
            model_response = extract_message_oai(response_data)
            return model_response
        else:
            return {"statusCode": response.status_code, "message": "Error: Unable to fetch recommendation"}
            
# def get_recommendation(genre, movies_df):
#     url = "https://llm.api.ai8.io/query_llm"
#     api_key = 'iBaXjSsgwr5umhLoAEa5BO1ZLhy62JIyMxH-x9ebG6w'
#     model = "gpt-3.5-turbo-0125"

#     # Fetch movies based on the specified genre
#     filtered_movies = movies_df[movies_df['genres_x'].str.contains(genre.capitalize())]

#     if len(filtered_movies) == 0:
#         return "No movies found for the specified genre."
#     else:
#         random_movie = filtered_movies.sample(n=1)
#         movie_to_recommend = random_movie['movie_title'].iloc[0]

#         # Define the data payload for OpenAI completion
#         data = {
#             "model": model,
#             "messages": [
#                 {"role": "system", "content": "You act as a highly intelligent system. Your job is to analyze the movie data and predict the rating scores of movies considering various movie features"},
#                 {"role": "user", "content": f"If I like {genre} movies from the dataset, recommend me a movie."}
#             ]
#         }

#         # Define headers with Authorization
#         headers = {'Authorization': api_key}

#         # Make a POST request to the OpenAI API
#         response = requests.post(url, json=data, headers=headers)

#         # Check if the request was successful (status code 200)
#         if response.status_code == 200:
#             # Extract response data
#             response_data = json.loads(response.content)
#             model_response = extract_message_oai(response_data)  # Define this function according to your needs
#             return model_response
#         else:
#             return {"statusCode": response.status_code, "message": "Error: Unable to fetch recommendation"}


In [ ]:
# def get_recommendation(genre):
#     filtered_movies = movies_df[movies_df['genres_x'].str.contains(genre.capitalize())]

#     if len(filtered_movies) == 0:
#         return "No movies found for the specified genre."
#     else:
#         random_movie = filtered_movies.sample(n=1)
#         movie_to_recommend = random_movie['movie_title'].iloc[0]

#         recommendation_prompt = f"If you like {genre} movies, you might also enjoy {other_movie_title}."

#         response = openai.Compeltion.create(
#             engine = "text-davinci-003",
#             prompt = recommendation_prompt, 
#             temperature = 0.7
#             max_toxens = 150 
#         )

#         try: 
#             generated_text = response['choices'][0]['text']
#             return generated_text

#         except Exception as e:
#             return f"Error: {e}"



# def get_recommendation(genre):
#     filtered_movies = movies_df[movies_df['genres_x'].str.contains(genre.capitalize())]

#     if len(filtered_movies) == 0:
#         return "No movies found for the specified genre."
#     else:
#         random_movie = filtered_movies.sample(n=1)
#         movie_to_recommend = random_movie['movie_title'].iloc[0]

#         recommendation_prompt = f"If I like {genre} movies from the dataset, recommend me a movie."

#         input_ids = tokenizer.encode(recommendation_prompt, return_tensors="pt").to(device)
#         attention_mask = torch.ones(input_ids.shape, device=device)
#         generated_output = model.generate(input_ids, attention_mask=attention_mask, max_length=150, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
#         generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)

#         return generated_text

In [51]:
user_genre = input("Please enter a genre: ")

Please enter a genre:  Drama


In [52]:
recommendation = get_recommendation(user_genre, movies_df)
print(recommendation)

Response data: b'{"id":"chatcmpl-9JIlJVfMANVVwndibpBVSfoXoJLyJ","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"I recommend the movie \\"The Shawshank Redemption.\\" It is a classic drama film that is highly acclaimed by critics and audiences alike. The movie tells the story of a banker who is wrongly convicted of murder and his experiences in Shawshank State Penitentiary. It is a powerful and emotional film that deals with themes of hope, redemption, and the resilience of the human spirit.","role":"assistant","function_call":null,"tool_calls":null}}],"created":1714387381,"model":"gpt-3.5-turbo-0125","object":"chat.completion","system_fingerprint":"fp_3b956da36b","usage":{"completion_tokens":78,"prompt_tokens":25,"total_tokens":103}}'
I recommend the movie "The Shawshank Redemption." It is a classic drama film that is highly acclaimed by critics and audiences alike. The movie tells the story of a banker who is wrongly convicted of murder and his exper